In [14]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import re
from nltk.stem import WordNetLemmatizer
import seaborn as sns
import pdb
from scipy.stats import truncnorm
import tensorflow as tf
import collections

In [15]:
config = {
    "emb_dim":300
    ,"maxWords": 30
    , "minWordFreq":10
    , "maxWordFreq": 10000
    , "WeightMultiplierPosClass": 11
}

In [ ]:
#sns.distplot(list(wordCount.values()))
data = np.array(list(wordCount.values()))
index = list(np.where(np.array(list(wordCount.values())) <= 10)[0])
#list(wordCount.keys())[8]
data[index]

In [13]:
tmp = pd.DataFrame({"ID":[1,2],"Id1":[3,4]})

for index,item in enumerate(tmp.iterrows()):
    #item[1]["ID"] = 10
    tmp.iloc

GloveEmbeddingsIndex[":"]

NameError: name 'GloveEmbeddingsIndex' is not defined

In [16]:
def cleanseDataset(data,IS_TRAIN):
    global stop_words,config
    
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    lemmatizer = WordNetLemmatizer()
    data["question_text"] = (data["question_text"].str.replace(u'\xa0', u' ')
                             .replace("n't", ' not ')
                             .replace("n's", '')
                             .replace(u'.', ' eos ')
                            )
    #data["question_text"] = data["question_text"].str.replace("n't", ' not ')
    #data["question_text"] = data["question_text"].str.replace("n's", '')
    #data["question_text"] = data["question_text"].str.replace(u'.', ' eos ')
    data["wordList"] = data["question_text"].apply(lambda x: re.split('\W+', x))
    data["wordList"] = data["wordList"].apply(lambda x: [lemmatizer.lemmatize(str.lower(word))
                                                     for word in x 
                                                     if (str.lower(word) not in stop_words) & (len(word) > 1)])
    data["countWords"] = data["wordList"].apply(lambda x: len(x))

    return data

def removeWordsLessThanThreshold(data,wordCount):
    global config
    
    wordListWithLessFreq = []
    columnList = []
    #---------Make a list of those words which are present less than specified threshold
    minWordFreq = config["minWordFreq"]
    maxWordFreq = config["maxWordFreq"]
    index = 0
    for row in data.iterrows():
        newWordList = []
        wordList = row[1]["wordList"]
        #pdb.set_trace()
        for word in wordList:
            if (wordCount[word] > minWordFreq) & (wordCount[word] < maxWordFreq):
                newWordList.append(word)
        columnList.append(newWordList)
        #row[1]["wordList"] = newWordList
        #pdb.set_trace()
        index += 1
        if (index % 1000000) == 0:
            print("Index {}".format(index))
    
    data["wordList"] = columnList
    data["countWords"] = data["wordList"].apply(lambda x: len(x))
    
    return data

def makeFixLengthWordVector(data,IS_TRAIN):
    global config
    
    maxWordsPerStatement = config["maxWords"]
    
    if IS_TRAIN:
        data = data[~((data["countWords"] < 1) | (data["countWords"] > maxWordsPerStatement))]
    
    #--------Make sure entire statement is of 30 words
    data["wordList"] = data["wordList"].apply(lambda x: 
                                             ["zerovec"] * ((maxWordsPerStatement - len(x)) if (maxWordsPerStatement > len(x)) else 0)
                                             + x[0:min(len(x),maxWordsPerStatement)]
                                             )
    return data
    
def getWordIndexMappingDictionary(wordCount):
    import collections
    
    uniqueWords = set(list(wordCount.keys()) + ["zerovec","eos"])
    
    word2idx = {}
    idx2word = {}
    def assignWords_Ids(word,index):
        word2idx[word] = index
        idx2word[index] = word
        
    for index,word in enumerate(uniqueWords):
        assignWords_Ids(word,index)
            
    return word2idx,idx2word

def assignIdsToWords(data,word2idx):
    data["WordIDList"] = data["wordList"].apply(lambda x: [word2idx[word] for word in x])
    return data

def loadEmbeddings(embeddingfile,word2idx):
    global config
    
    GloveEmbeddingsIndex = {}
    emb_dim = config["emb_dim"]
    fe = open(embeddingfile,"r",encoding="utf-8",errors="ignore")
    for index, line in enumerate(fe):
        tokens= line.strip().split()
        word = tokens[0]
        try:
            vec = list(map(float,tokens[1:]))
            if word in word2idx:
                GloveEmbeddingsIndex[word]=vec
        except:
            print("Exception occured: " + str(index))
            
    #Add Zerovec, this will be useful to pad zeros, it is better to experiment with padding any non-zero constant values also.
    #GloveEmbeddings["zerovec"] = "0.0 "*emb_dim
    GloveEmbeddingsIndex["zerovec"] = [0.0] *emb_dim
    GloveEmbeddingsIndex["eos"] = [0.0] *emb_dim
    fe.close()
    return GloveEmbeddingsIndex
    
def getWordsCollection(train,test):
    trainWords = [a for b in train["wordList"].tolist() for a in b]
    testWords = [a for b in test["wordList"].tolist() for a in b]
    words = trainWords + testWords
    wordCount = collections.Counter(words)
    return wordCount

def assignEmbeddingsToWords(word2idx,GloveEmbeddingsIndex):
    emb_dim = config["emb_dim"]
    vocab_size = len(word2idx)
    GloveEmbeddings = np.zeros((vocab_size,emb_dim))
    for key,value in word2idx.items():
        if(key not in GloveEmbeddingsIndex):
            GloveEmbeddings[value] = np.random.normal(0,.1,emb_dim)
        else:
            GloveEmbeddings[value] = GloveEmbeddingsIndex[key]
    return GloveEmbeddings

In [17]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
train = cleanseDataset(train,True)
test = cleanseDataset(test,False)
wordCount = getWordsCollection(train,test)

In [26]:
train.iloc[np.where(train["question_text"].apply(lambda x: ":" in x))]
#print(train.iloc[15655]["question_text"])

,qid,question_text,target,wordList,countWords
475,001689a792974397f88f,Why did my clock stop at 3:00 in the morning?,0,"[clock, stop, 00, morning]",4
525,00190731e5d576d93154,Who is likely to be fluent in English: a Ghana...,0,"[likely, fluent, english, ghanaian, nigerian]",5
999,0031884048d46eea1e7f,Who will win in video game battle: Adam d ange...,1,"[win, video, game, battle, adam, angelo, v, ki...",9
1254,003df1cd16501c899cff,How does your personal knowledge and experienc...,0,"[personal, knowledge, experience, affect, way,...",14
1462,004913683f771ac568d3,How does your personal knowledge and experienc...,0,"[personal, knowledge, experience, affect, way,...",14
1564,004e76ae4069b9632dd0,Parents: Have you ever temporarily lost your y...,0,"[parent, ever, temporarily, lost, young, child...",10
1574,004f17a44eddd8f0a038,If you had to vote between: Mortal Kombat and ...,0,"[vote, mortal, kombat, grand, theft, auto, wou...",8
1819,005a84aff2a194793a0c,Whats the possible causes of the following beh...,0,"[whats, possible, cause, following, behavior, ...",14
2154,006ad579fe95ac7955c8,"What is the ""Maximum Ride: School's Out Foreve...",0,"[maximum, ride, school, forever, book, james, ...",7
2339,0074d1015a795543628e,"""Car dry cleaning & car cleaning and detailing...",0,"[car, dry, cleaning, car, cleaning, detailing,...",16


In [7]:
train = removeWordsLessThanThreshold(train,wordCount)
test = removeWordsLessThanThreshold(test,wordCount)
wordCount = getWordsCollection(train,test)
word2idx, idx2word = getWordIndexMappingDictionary(wordCount)
train = makeFixLengthWordVector(train,True)
test = makeFixLengthWordVector(test,False)

Index 1000000


C:\Users\sitandon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
GloveEmbeddingsIndex = loadEmbeddings("./Data/embeddings/glove.840B.300d/glove.840B.300d.txt",word2idx)
GloveEmbeddings = assignEmbeddingsToWords(word2idx,GloveEmbeddingsIndex)

#del GloveEmbeddingsIndex,wordCount

KeyboardInterrupt: 

In [48]:
train = assignIdsToWords(train,word2idx)
test = assignIdsToWords(test,word2idx)

In [ ]:
df = pd.DataFrame({'id':['a','b', 'c'], 'val':[['val1','val2','val33'],
                                               ['val33','val9','val6'],
                                               ['val2','val6','val7']]})

(list(set([a for b in df.val.tolist() for a in b])))

In [49]:
train = train[["WordIDList","target"]]
test = test[["WordIDList"]]

In [50]:
def model_inputs():
    """
    Create the model inputs
    """
    
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob_ = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs_, labels_, keep_prob_

def build_embedding_layer(inputs_, vocab_size, embed_size):
    """
    Create the embedding layer
    """
    global GloveEmbeddings
    
    glove_weights_initializer = tf.constant_initializer(GloveEmbeddings)
    embedding = tf.get_variable(
                name='embedding_weights', 
                shape=(vocab_size, embed_size), 
                initializer=glove_weights_initializer,
                trainable=True
            )
    
    embed = tf.nn.embedding_lookup(embedding, inputs_)
    
    return embed


def build_lstm_layers(lstm_sizes, embed, keep_prob_, batch_size):
    """
    Create the LSTM layers
    """
    lstms = [tf.contrib.rnn.BasicLSTMCell(size) for size in lstm_sizes]
    # Add dropout to the cell
    drops = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_) for lstm in lstms]
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell(drops)
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    lstm_outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)
    
    return initial_state, lstm_outputs, cell, final_state

def build_cost_fn_and_opt(lstm_outputs, labels_, learning_rate):
    """
    Create the Loss function and Optimizer
    """
    tmpMultiplier = config["WeightMultiplierPosClass"]
    weights = tf.multiply(tmpMultiplier, tf.cast(tf.argmax(labels_,-1),tf.int32)) + 1
    predictions = tf.contrib.layers.fully_connected(lstm_outputs[:, -1], 2, activation_fn=None)
    loss = tf.losses.softmax_cross_entropy(onehot_labels = labels_, logits = predictions, weights = weights)
    optimzer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
    
    return predictions, loss, optimzer


def build_accuracy(predictions, labels_):
    """
    Create accuracy
    """
    correct_pred = tf.equal(tf.argmax(predictions), tf.argmax(labels_))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.int32))
    
    return accuracy

def getBatches(data, batch_size):
    
    L = len(data)
        
    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
            
            X = np.concatenate(data.iloc[batch_start:limit]["WordIDList"].values,axis = 0).reshape(-1,config["maxWords"])
            Y = np.array([[1,0] if item[0] == 0 else [0,1] for item in data.iloc[batch_start:limit][["target"]].values])
            #data.iloc[batch_start:limit][["target"]].values
            
            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size

In [51]:
def test_network(model_dir, batch_size, test_x, test_y):
    
    inputs_, labels_, keep_prob_ = model_inputs()
    embed = build_embedding_layer(inputs_, vocab_size, embed_size)
    initial_state, lstm_outputs, lstm_cell, final_state = build_lstm_layers(lstm_sizes, embed, keep_prob_, batch_size)
    predictions, loss, optimizer = build_cost_fn_and_opt(lstm_outputs, labels_, learning_rate)
    accuracy = build_accuracy(predictions, labels_)
    
    saver = tf.train.Saver()
    
    test_acc = []
    with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint(model_dir))
        test_state = sess.run(lstm_cell.zero_state(batch_size, tf.float32))
        for ii, (x, y) in enumerate(utl.get_batches(test_x, test_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob_: 1,
                    initial_state: test_state}
            batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
            test_acc.append(batch_acc)
        print("Test Accuracy: {:.3f}".format(np.mean(test_acc)))

In [2]:
def build_and_train_network(lstm_sizes, vocab_size, embed_size, epochs, batch_size, 
                            learning_rate, keep_prob, train, val):
    
    inputs_, labels_, keep_prob_ = model_inputs()
    embed = build_embedding_layer(inputs_, vocab_size, embed_size)
    initial_state, lstm_outputs, lstm_cell, final_state = build_lstm_layers(lstm_sizes, embed, keep_prob_, batch_size)
    predictions, loss, optimizer = build_cost_fn_and_opt(lstm_outputs, labels_, learning_rate)
    accuracy = build_accuracy(predictions, labels_)
    
    saver = tf.train.Saver(max_to_keep=2)
    
    best_acc = 0
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        n_batches = len(train)//batch_size
        trainGenerator = getBatches(train,batch_size)
        validGenerator = getBatches(val,len(val))
        for e in range(epochs):
            state = sess.run(initial_state)
            
            train_acc = []
            lossValue = []
            for ii in range(n_batches):
                (x,y) = next(trainGenerator)
                #print(y.shape)
                #print(y)
                feed = {inputs_: x,
                        labels_: y,
                        keep_prob_: keep_prob,
                        initial_state: state}
                loss_, state, _,  batch_acc = sess.run([loss, final_state, optimizer, accuracy], feed_dict=feed)
                train_acc.append(batch_acc)
                lossValue.append(loss_)
                
                if (ii + 1) % 100 == 0:
                    print("{}/{} Train Accuracy: {}, Loss: {}".format(e+1,ii+1, np.mean(train_acc),np.mean(lossValue)))
                        
                if ((ii + 1) % n_batches) == 0:
                    
                    val_acc = []
                    val_state = sess.run(lstm_cell.zero_state(batch_size, tf.float32))
                    
                    #for index in utl.get_batches(val_x, val_y, batch_size):
                    (xx,yy) = next(validGenerator)
                    feed = {inputs_: xx,
                            labels_: yy,
                            keep_prob_: 1,
                            initial_state: val_state}
                    val_batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(val_batch_acc)
                    
                    acc = np.mean(val_acc)
                    print("Epoch: {}/{}...".format(e+1, epochs),
                          "Batch: {}/{}...".format(ii+1, n_batches),
                          "Train Loss: {:.3f}...".format(loss_),
                          "Train Accruacy: {:.3f}...".format(np.mean(train_acc)),
                          "Val Accuracy: {:.3f}".format(acc))
                
                    if best_acc < acc:
                        best_acc = acc
                        saver.save(sess, "./checkpoints/sentiment.ckpt")

In [53]:
from sklearn.model_selection import train_test_split
train,val = train_test_split(train,test_size = .2)
train = train.reset_index(drop = True)
val = val.reset_index(drop = True)

In [ ]:
# Define Inputs and Hyperparameters
lstm_sizes = [128,64]
vocab_size = len(word2idx)  #add one for padding
embed_size = config["emb_dim"]
epochs = 10
batch_size = 64
learning_rate = 0.001
keep_prob = 0.5

with tf.Graph().as_default():
    build_and_train_network(lstm_sizes, vocab_size, embed_size, epochs, batch_size,
                            learning_rate, keep_prob, train,val)

1/100 Train Accuracy: 0.05000000074505806...
1/200 Train Accuracy: 0.04749999940395355...
1/300 Train Accuracy: 0.05999999865889549...
1/400 Train Accuracy: 0.06499999761581421...
1/500 Train Accuracy: 0.07400000095367432...
1/600 Train Accuracy: 0.07916666567325592...
1/700 Train Accuracy: 0.07999999821186066...
1/800 Train Accuracy: 0.08250000327825546...
1/900 Train Accuracy: 0.08166666328907013...
1/1000 Train Accuracy: 0.08250000327825546...
1/1100 Train Accuracy: 0.08500000089406967...
1/1200 Train Accuracy: 0.08416666835546494...
1/1300 Train Accuracy: 0.08576922863721848...
1/1400 Train Accuracy: 0.0860714316368103...
1/1500 Train Accuracy: 0.08699999749660492...
1/1600 Train Accuracy: 0.0846875011920929...
1/1700 Train Accuracy: 0.0847058817744255...
1/1800 Train Accuracy: 0.0855555534362793...
1/1900 Train Accuracy: 0.08605263382196426...
1/2000 Train Accuracy: 0.08799999952316284...
1/2100 Train Accuracy: 0.08595237880945206...
1/2200 Train Accuracy: 0.08636363595724106...
1

In [ ]:
#np.concatenate(train.loc[5:10]["WordIDList"].values,axis = 0).shape
#np.array([[1,0] if item[0] == 0 else [0,1] for item in train[["target"]].head(12).values]).shape
train.head()

In [ ]:
with tf.Graph().as_default():
    test_network('checkpoints', batch_size, test_x, test_y)`